# <span style="color:blue">Bulletproofing Code</span>

## What could go wrong? Unfortunately, lots!

![](images/worst_thing_that_could_happen.png)

## <span style="color:darkorange">Potential Problems: </span>
- Bugs (code crashes, brittle to unexpected inputs)
- Code "works", but gives incorrect results
- Cannot reliably and automatically generate the same results each time
- External resources, like code dependencies and data change outside your control
- Code is slow and/or uses a lot of memory
- Your code is hard to understand
- Your code is hard to change

## <span style="color:darkorange">Make your code work first before trying to optimize it</span>

![](images/knuth.jpg)

# <span style="color:blue">Example: Extracting Information from Earnings Calls</span>

In [ ]:
from pathlib import Path
import json

In [ ]:
base_path = Path("../tests/data/ciq_transcript_samples")

In [ ]:
sample_file = base_path / "Tesla/transcript_id__1025482.json"
text = sample_file.read_text()
transcript_dict = json.loads(text)
components = transcript_dict["components"]
tesla_answers = [c for c in components if c["componenttypename"] == "Answer"]

In [ ]:
sample_file = base_path / "GM/transcript_id__1000656.json"
text = sample_file.read_text()
transcript_dict = json.loads(text)
components = transcript_dict["components"]
gm_answers = [c for c in components if c["componenttypename"] == "Answer"]

In [ ]:
for answer in tesla_answers:
    text = answer['text']
text

In [ ]:
import textacy
doc = textacy.make_spacy_doc(text)

# <span style="color:blue">Example: Calculating Document Similarity</span>

# <span style="color:blue">Testing, Error Detection, and Profiling</span>

![](images/165-minor-change.png)

## Testing

In [ ]:
# dot product using for-loop

## Exception Handling

## Profiling

https://docs.python.org/3/library/debug.html

In [ ]:
# dot product using for-loop

## Configuring your Code Dependencies

Conda environments are cheap to create and easy to delete

In [ ]:
! conda env list

Notice how many packages there are, so many opportunities for something to change and potentially break your code! If you're using a package, try to find ones with a sizable support community, not one-offs from an undergraduate class project.

In [ ]:
! conda list

Tip: export your (pinned) dependencies to a file. You can use this to re-create your environment reproducibly, anwhere, and any number of times.

In [41]:
! conda env export --from-history

name: workshop-env
channels:
  - conda-forge
  - defaults
dependencies:
  - python==3.7
  - spacy
  - textacy
  - jupyterlab
  - sqlalchemy
  - pytest
  - pyyaml
prefix: /Users/willthompson/anaconda3/envs/workshop-env



In [44]:
! conda env export --from-history | grep -v "^prefix: " > environment.yml
! sed -i '' 's/workshop-env/test-env/g' environment.yml

In [45]:
! conda env create -f environment.yml

Solving environment: done
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
#
# To activate this environment, use
#
#     $ conda activate test-env
#
# To deactivate an active environment, use
#
#     $ conda deactivate



In [46]:
! conda env list

# conda environments:
#
base                     /Users/willthompson/anaconda3
aws                      /Users/willthompson/anaconda3/envs/aws
cafrs                    /Users/willthompson/anaconda3/envs/cafrs
cluedo                   /Users/willthompson/anaconda3/envs/cluedo
corelogic                /Users/willthompson/anaconda3/envs/corelogic
edx                      /Users/willthompson/anaconda3/envs/edx
gcp                      /Users/willthompson/anaconda3/envs/gcp
mturk                    /Users/willthompson/anaconda3/envs/mturk
ocr-review               /Users/willthompson/anaconda3/envs/ocr-review
patent-data              /Users/willthompson/anaconda3/envs/patent-data
pytorch-nlp              /Users/willthompson/anaconda3/envs/pytorch-nlp
test-env                 /Users/willthompson/anaconda3/envs/test-env
textract                 /Users/willthompson/anaconda3/envs/textract
voilatest                /Users/willthompson/anaconda3/envs/voilatest
workshop-env          *  /Users/willt

In [47]:
! conda env remove -n test-env


Remove all packages in environment /Users/willthompson/anaconda3/envs/test-env:



## Configuring the Entire Environment

![](images/horizontal-logo-monochromatic-white.png)

![](images/container-what-is-container.png)

https://www.docker.com/

In [48]:
! docker build .. -t workshop:latest
# Open terminal to docker image: docker run -i -t workshop:latest /bin/bash

Sending build context to Docker daemon  10.83MB
Step 1/6 : FROM continuumio/miniconda3
 ---> 406f2b43ea59
Step 2/6 : ADD environment.yml /tmp/environment.yml
 ---> Using cache
 ---> 39c150d01db3
Step 3/6 : RUN conda update -n base -c defaults conda
 ---> Using cache
 ---> 948702393f04
Step 4/6 : RUN conda env create -f /tmp/environment.yml
 ---> Using cache
 ---> 3cfac15cf81d
Step 5/6 : RUN echo "source activate $(head -1 /tmp/environment.yml | cut -d' ' -f2)" > ~/.bashrc
 ---> Using cache
 ---> 57ee8eb6cc1a
Step 6/6 : ENV PATH /opt/conda/envs/$(head -1 /tmp/environment.yml | cut -d' ' -f2)/bin:$PATH
 ---> Using cache
 ---> b09c038006cb
Successfully built b09c038006cb
Successfully tagged workshop:latest
